In [1]:

using Flux
using Flux: onehot, chunk, batchseq, throttle, crossentropy
using StatsBase: wsample
using Base.Iterators: partition

cd(@__DIR__)

# isfile("input.txt") ||
#   download("https://cs.stanford.edu/people/karpathy/char-rnn/shakespeare_input.txt",
#            "input.txt")

text = collect(String(read("input.txt")))
alphabet = [unique(text)..., '_']
text = map(ch -> onehot(ch, alphabet), text)
stop = onehot('_', alphabet)

N = length(alphabet)
seqlen = 50
nbatch = 50

Xs = collect(partition(batchseq(chunk(text, nbatch), stop), seqlen))
Ys = collect(partition(batchseq(chunk(text[2:end], nbatch), stop), seqlen))

m = Chain(
  LSTM(N, 128),
  LSTM(128, 128),
  Dense(128, N),
  softmax)

m = gpu(m)

function loss(xs, ys)
  l = sum(crossentropy.(m.(gpu.(xs)), gpu.(ys)))
  Flux.truncate!(m)
  return l
end

opt = ADAM(params(m),0.01)
tx, ty = (Xs[5], Ys[5])
evalcb = () -> @show loss(tx, ty)

Flux.train!(loss, zip(Xs, Ys), opt,
            cb = throttle(evalcb, 30))

# Sampling

function sample(m, alphabet, len; temp = 1)
  m = cpu(m)
  Flux.reset!(m)
  buf = IOBuffer()
  c = rand(alphabet)
  for i = 1:len
    write(buf, c)
    c = wsample(alphabet, m(onehot(c, alphabet)).data)
  end
  return String(take!(buf))
end

sample(m, alphabet, 1000) |> println

# evalcb = function ()
#   @show loss(Xs[5], Ys[5])
#   println(sample(deepcopy(m), alphabet, 100))
# end

loss(tx, ty) = 200.47934676994632 (tracked)
loss(tx, ty) = 161.208454060423 (tracked)
loss(tx, ty) = 139.6480230504045 (tracked)
loss(tx, ty) = 125.85372934360022 (tracked)
loss(tx, ty) = 120.14623446463074 (tracked)
loss(tx, ty) = 115.63402455734291 (tracked)
loss(tx, ty) = 113.84868300504844 (tracked)
loss(tx, ty) = 111.1286469817872 (tracked)
loss(tx, ty) = 108.33348286614869 (tracked)
loss(tx, ty) = 106.50256946850341 (tracked)
loss(tx, ty) = 104.83230962877253 (tracked)
loss(tx, ty) = 102.75439118793871 (tracked)
loss(tx, ty) = 103.30092014920228 (tracked)
loss(tx, ty) = 101.27294965234854 (tracked)
loss(tx, ty) = 99.10074812471943 (tracked)
loss(tx, ty) = 99.49734229402746 (tracked)
loss(tx, ty) = 98.73482729200897 (tracked)
loss(tx, ty) = 97.68629172174403 (tracked)
loss(tx, ty) = 97.43084663275992 (tracked)
loss(tx, ty) = 97.47560285410557 (tracked)
loss(tx, ty) = 97.24012850995815 (tracked)
loss(tx, ty) = 97.68119960343672 (tracked)
loss(tx, ty) = 95.63200684394556 (tracked)
l

In [4]:
length(text)

4573338